## Create an experiment in workspace

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()

#Create an experiment
exp = Experiment(workspace=ws, name="Optimizing-Pipelines")
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')
run = exp.start_logging()

Workspace name: quick-starts-ws-133800
Azure region: southcentralus
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-133800


## Create a compute cluster

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

#Create a new cluster to run the experiment using vm_size of STANDARD_D2_V2 and max_nodes of 4. 
cpu_cluster_name = "cpucluster"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found an existing cluster.\n")
except ComputeTargetException:
    print("Creating a new cluster.\n")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion()

#If the cluster exist already for the current cluster, print it's detailed status using get_status(). If not, create and print.
print("Cluster details: ", cpu_cluster.get_status().serialize())

Found an existing cluster.

Cluster details:  {'currentNodeCount': 4, 'targetNodeCount': 4, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 4, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-01-07T10:16:15.692000+00:00', 'errors': None, 'creationTime': '2021-01-07T10:11:59.744128+00:00', 'modifiedTime': '2021-01-07T10:12:15.406265+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Scikit-learn based logistic regression

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
import os

#Specify a parameter sampler (Random sampling)
ps = RandomParameterSampling({
       '--C': uniform(0.01,1),
        '--max_iter': choice(100, 150, 200, 250, 300)})

#Specify a policy (Bandit policy)
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

#Create a directory 'training'
if "training" not in os.listdir():
    os.mkdir("./training")

#Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./',
                compute_target=cpu_cluster,
                entry_script='train.py')

#Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
                                   hyperparameter_sampling = ps,
                                   primary_metric_name = 'accuracy',
                                   primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                   max_total_runs = 20,
                                   max_concurrent_runs = 4,
                                   policy = policy,
                                   estimator = est)


'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


Though SKLearn is deprecated, it works fine in this case. 

In [4]:
#Submit the hyperdrive run
hd_run = exp.submit(hyperdrive_config)

#launch the widget to view the progress and results
RunDetails(hd_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [5]:
import joblib
from azureml.core.model import Model

#Get the best run and save the model from that run.
best_run_hd = hd_run.get_best_run_by_primary_metric()
model_hd = best_run_hd.register_model(model_name='hyperdrive_best_model', 
                                model_path='./outputs/model.pkl',
                                model_framework=Model.Framework.SCIKITLEARN, 
                                model_framework_version='0.19.1')
print("Model successfully saved.")

Model successfully saved.


## AutoML

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create a TabularDataset 'ds' using TabularDatasetFactory
ds = TabularDatasetFactory.from_delimited_files(path = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')

In [7]:
from train import clean_data
import pandas as pd
from azureml.core import Dataset
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

#Split the dataset into train and test dataset. Combine x_train and y_train. 
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)
df_train = pd.concat([x_train,y_train], axis=1)

#Convert x_train and y_train (Which are in pandas DataFrame format) to TabularDataset format.
try:
    os.makedirs('./data', exist_ok=True)
except OSError as error:
    print('New directory cannot be created')
    
path = 'data/train.csv'
df_train.to_csv(path)

datastore = ws.get_default_datastore()
datastore.upload(src_dir='data', target_path='data')

train_data = TabularDatasetFactory.from_delimited_files(path=[(datastore, ('data/train.csv'))])
print("Successfully converted the dataset to TabularDataset format.")

Uploading an estimated of 1 files
Uploading data/train.csv
Uploaded data/train.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Successfully converted the dataset to TabularDataset format.


In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_data,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=cpu_cluster)

In [9]:
#Run AutoML on a remote compute target
remote_run = exp.submit(config=automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on cpucluster with default configuration
Running on remote compute: cpucluster
Parent Run ID: AutoML_73570c30-33e7-421f-8b4f-9c0a325f547e

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------

In [10]:
#launch the widget to view the progress and results
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [11]:
# Retrieve and save the best automl model.
best_run_AutoML, fitted_model_AutoML = remote_run.get_output()
model_AutoML = best_run_AutoML.register_model(model_path='./outputs/', model_name='Output_automl.pkl')
print("Model saved successfully")

Model saved successfully


## Comparing the models

In [13]:
#HyperDrive
print("Scikit-learn based Logistic regression: ")
best_run_metrics_hd = best_run_hd.get_metrics()
print("Best Run Id: ", best_run_hd.id)
print("Accuracy: ", best_run_metrics_hd['accuracy'])

#AutoML
print("\n\nAutoML:")
best_run_metrics_AutoML = best_run_AutoML.get_metrics()
print("Best run Id: ",best_run_AutoML.id)
print("Accuracy: ", best_run_metrics_AutoML['accuracy'])
print("Other details: ")
best_run_AutoML.get_tags()
#print("\nFitted model:\n",fitted_model_AutoML)

Scikit-learn based Logistic regression: 
Best Run Id:  HD_dd2413a4-6002-4dfd-b2d1-8620a74ffa4a_12
Accuracy:  0.9153262518968134


AutoML:
Best run Id:  AutoML_73570c30-33e7-421f-8b4f-9c0a325f547e_24
Accuracy:  0.9149468892261001
Other details: 


{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":0,"CurrentNodeCount":1}',
 'ensembled_iterations': '[0, 1, 20, 23, 9, 4, 21]',
 'ensembled_algorithms': "['LightGBM', 'XGBoostClassifier', 'RandomForest', 'LightGBM', 'LogisticRegression', 'RandomForest', 'ExtremeRandomTrees']",
 'ensemble_weights': '[0.35714285714285715, 0.14285714285714285, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.21428571428571427]',
 'best_individual_pipeline_score': '0.9134673748103186',
 'best_individual_iteration': '0',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True'}